# Taller 06: Bases de Datos Vectoriales  
**ICCD753 Recuperación de Información – Prof. Iván Carrera (2024-B, EPN-FIS)**  

**Fecha de Entrega:** Martes 14 de enero de 2025  
**Integrantes:**  
- Dilan Andrade  
- Hernán Sánchez  
- Galo Tarapués  

---

### **Objetivo del Taller**
Implementar y comparar estrategias de recuperación de información utilizando TF-IDF, BM25 y embeddings con bases de datos vectoriales, evaluando su relevancia y eficiencia en un dataset textual real.


### **Parte 1: Importación de Librerías para TF-IDF**
En esta celda se importan las librerías fundamentales para la implementación de la recuperación de información utilizando el modelo TF-IDF. Estas librerías permiten:

1. **`pandas`**: Manejar y procesar el dataset estructurado del taller.
2. **`TfidfVectorizer`** (de `scikit-learn`): Transformar los textos en una matriz TF-IDF, donde cada documento es representado como un vector en un espacio de palabras.
3. **`cosine_similarity`** (de `scikit-learn`): Calcular la similitud entre vectores de documentos y consultas, para determinar qué tan relevante es un documento en relación con la consulta.


In [7]:

# Importar librerías necesarias

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


### **Parte 2: Carga y Preprocesamiento del Dataset**
En esta celda se realiza la carga y preprocesamiento inicial del dataset `Wikipedia Movie Plots`. Los pasos incluyen:

1. **Definir la ruta del dataset**: Se especifica la ubicación del archivo CSV (`wiki_movie_plots_deduped.csv`).
2. **Cargar el dataset**: Se utiliza `pandas` para cargar el archivo CSV en un DataFrame, que permite manipular y analizar los datos.
3. **Eliminación de filas inválidas**: Se eliminan las filas que no contienen información válida en la columna `Plot`, ya que esta información es esencial para la tarea de recuperación.
4. **Verificación de datos**: Se muestran las primeras filas del DataFrame para asegurarse de que la carga y el preprocesamiento sean correctos.




In [8]:
ruta_dataset = "wiki_movie_plots_deduped.csv"

df = pd.read_csv(ruta_dataset)

df = df.dropna(subset=['Plot'])

df.head()


,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...


### **Parte 3: Combinación de Título y Trama para el Modelo TF-IDF**
En esta celda se integra la información del título y la trama de cada película en una nueva columna llamada `texto_completo`. Este paso es crucial para que el modelo TF-IDF considere tanto el título como el contenido de la trama durante la extracción de características. 

#### Detalles clave:
1. **Integración de datos**: Se crea la columna `texto_completo` combinando el título (`Title`) y la trama (`Plot`) de cada película.
2. **Posibilidad de preprocesamiento adicional**:
   - Conversión a minúsculas para normalizar el texto.
   - Aplicación de expresiones regulares, lematización o stemming para mejorar la calidad del texto procesado.




In [9]:

df['texto_completo'] = df['Title'].astype(str) + " " + df['Plot'].astype(str)


### **Parte 4: Configuración y Generación de la Matriz TF-IDF**
En esta celda se configura y genera la matriz TF-IDF para representar los documentos del dataset como vectores numéricos, listos para ser utilizados en cálculos de similitud.

#### Detalles clave:
1. **Configuración del `TfidfVectorizer`**:
   - `stop_words='english'`: Elimina palabras comunes en inglés (stopwords) que no aportan valor semántico.
   - `max_features=20000`: Limita el vocabulario a las 20,000 palabras más importantes según la frecuencia.
   - `sublinear_tf=True`: Aplica una escala logarítmica a las frecuencias para reducir el impacto de palabras extremadamente frecuentes.
   - `ngram_range=(1, 2)`: Captura unigramas (palabras individuales) y bigramas (pares de palabras consecutivas).
   
2. **Transformación de los documentos**:
   - Se ajusta el vectorizador a la columna `texto_completo`, generando una matriz TF-IDF.
   - Cada fila de la matriz representa un documento, y cada columna, una palabra o bigrama en el vocabulario.

3. **Salida**:
   - La forma de la matriz (`n_documentos`, `n_features`) es impresa para verificar la correcta generación.


In [10]:

vectorizador_tfidf = TfidfVectorizer(
    stop_words='english',
    max_features=20000,
    sublinear_tf=True,
    ngram_range=(1, 2)
)

# Ajustamos el vectorizador a la nueva columna 'texto_completo'
matriz_tfidf = vectorizador_tfidf.fit_transform(df['texto_completo'])

print("Matriz TF-IDF generada con forma:", matriz_tfidf.shape)


Matriz TF-IDF generada con forma: (34886, 20000)


### **Parte 5: Función para Realizar Búsquedas con TF-IDF**
Esta celda define una función que permite realizar búsquedas en el dataset utilizando el modelo TF-IDF. Dada una consulta de texto, la función encuentra los documentos más relevantes mediante el cálculo de la similitud de coseno entre la consulta y los documentos del dataset.

#### Detalles clave:
1. **Transformación de la consulta**:
   - La consulta se transforma en un vector TF-IDF utilizando los mismos parámetros con los que se entrenó el vectorizador original.
   
2. **Cálculo de similitud**:
   - Se calcula la similitud de coseno entre el vector de la consulta y todos los documentos del dataset.
   
3. **Ordenamiento de resultados**:
   - Los documentos se ordenan en función de su puntuación de similitud en orden descendente.
   - Se retornan los índices de los documentos más relevantes y sus puntuaciones.

#### Parámetros:
- `consulta` (str): Texto de la consulta a buscar.
- `numero_resultados` (int): Número de documentos más relevantes a devolver (por defecto, 5).

#### Retorno:
- `indices_ordenados`: Índices de los documentos más relevantes.
- `puntuaciones`: Puntuaciones de similitud correspondientes a los documentos.


In [11]:

def buscar_con_tfidf(consulta, numero_resultados=5):
    # Convertimos la consulta al vector TF-IDF (mismos parámetros que el vectorizador entrenado)
    vector_consulta = vectorizador_tfidf.transform([consulta])

    # Calculamos similitud de coseno con todos los documentos
    similitudes = cosine_similarity(vector_consulta, matriz_tfidf).flatten()

    # Ordenamos los documentos por su puntuación de similitud (descendente)
    indices_ordenados = similitudes.argsort()[::-1][:numero_resultados]
    puntuaciones = similitudes[indices_ordenados]

    return indices_ordenados, puntuaciones


### **Parte 6: Ejemplo de Consulta con TF-IDF**
En esta celda se realiza una consulta de ejemplo utilizando la función `buscar_con_tfidf`, previamente definida. El propósito es recuperar los documentos más relevantes para una consulta específica y mostrar información útil sobre ellos.

#### Detalles clave:
1. **Consulta de ejemplo**:
   - La consulta `"dinosaurs"` se utiliza para probar el sistema de búsqueda.
   
2. **Recuperación de documentos**:
   - La función `buscar_con_tfidf` retorna los índices y las puntuaciones de los documentos más relevantes en el dataset.
   
3. **Visualización de resultados**:
   - Por cada documento relevante, se imprime:
     - **Rango**: Posición del documento en el ranking.
     - **Título**: El título de la película.
     - **Similitud**: Puntuación de similitud calculada con el modelo TF-IDF.
     - **Trama**: Los primeros 200 caracteres de la trama, para evitar saturar la salida.



In [15]:

# Ejemplo de consulta
consulta_ejemplo = "dinosaurs"

# Obtenemos los documentos más similares
indices_encontrados, puntuaciones_encontradas = buscar_con_tfidf(consulta_ejemplo, numero_resultados=5)

print(f"Resultados para la consulta: '{consulta_ejemplo}'\n")
for rank, (indice, puntuacion) in enumerate(zip(indices_encontrados, puntuaciones_encontradas), start=1):
    print(f"Rank {rank}")
    print(f"Título (Title): {df.loc[indice, 'Title']}")
    print(f"Similitud: {puntuacion:.4f}")
    # Mostramos los primeros 200 caracteres de la trama (Plot) para no saturar la salida
    print(f"Trama (Plot) (primeros 200 caracteres): {df.loc[indice, 'Plot'][:200]}...")
    print("-"*50)


Resultados para la consulta: 'dinosaurs'

Rank 1
Título (Title): Theodore Rex
Similitud: 0.3514
Trama (Plot) (primeros 200 caracteres): In an alternate futuristic society where humans and anthropomorphic dinosaurs co-exist, a tough police detective named Katie Coltraine (Whoopi Goldberg) is paired with a Tyrannosaurus named Theodore R...
--------------------------------------------------
Rank 2
Título (Title): Dinosaurs! – A Fun-Filled Trip Back in Time!
Similitud: 0.2623
Trama (Plot) (primeros 200 caracteres): The video—with beginning scenes filmed in 1987—begins with a young boy named Phillip (played by Fred Savage) sitting in his bedroom, listening to loud music, and struggling to find an idea for a class...
--------------------------------------------------
Rank 3
Título (Title): We're Back! A Dinosaur's Story
Similitud: 0.2593
Trama (Plot) (primeros 200 caracteres): In present-day New York City, an Eastern bluebird named Buster runs away from his siblings and he meets an intellige

## Reflexión Parte 1: Recuperación con TF-IDF
En esta parte, implementamos un modelo TF-IDF para calcular la relevancia de términos dentro del dataset de tramas de películas. El enfoque resultó efectivo para identificar documentos relacionados directamente con las palabras clave, aunque su capacidad es limitada frente a términos no exactos o con ambigüedades.


# Parte 2 Recuperación con BM25

In [13]:

from elasticsearch import Elasticsearch
import pandas as pd
from tqdm import tqdm
import time

In [14]:
# Conectar con Elasticsearch

# Configuramos la conexión con Elasticsearch
es = Elasticsearch(['http://localhost:9200'])

# Verificamos la conexión
if es.ping():
    print("Conectado a Elasticsearch")
else:
    print("No se pudo conectar a Elasticsearch")

Conectado a Elasticsearch


In [15]:
# Configuración del índice para almacenar los documentos
index_name = "movie_plots"

# Definir el esquema del índice con mapeos para BM25
index_config = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0,
        "analysis": {
            "analyzer": {
                "custom_analyzer": {
                    "type": "standard",
                    "stopwords": "_english_"
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "Title": {"type": "text", "analyzer": "custom_analyzer"},
            "Plot": {"type": "text", "analyzer": "custom_analyzer"}
        }
    }
}

# Crear el índice en Elasticsearch
if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name, body=index_config)
    print(f"Índice '{index_name}' creado exitosamente.")
else:
    print(f"El índice '{index_name}' ya existe.")


El índice 'movie_plots' ya existe.


In [20]:
# Cargar el dataset
ruta_dataset = "wiki_movie_plots_deduped.csv"
df = pd.read_csv(ruta_dataset)

# Filtrar filas con valores nulos en 'Plot'
df = df.dropna(subset=['Plot'])

# Insertar documentos en el índice
for i, row in tqdm(df.iterrows(), total=len(df), desc="Indexando documentos en Elasticsearch"):
    doc = {
        "Title": row['Title'],
        "Plot": row['Plot']
    }
    es.index(index=index_name, id=i, body=doc)

print(f"Se han indexado {len(df)} documentos en el índice '{index_name}'.")


Indexando documentos en Elasticsearch: 100%|██████████| 34886/34886 [31:55<00:00, 18.21it/s]  

Se han indexado 34886 documentos en el índice 'movie_plots'.


In [16]:
def buscar_con_bm25(consulta, numero_resultados=5):
    """
    Realiza una consulta al índice en Elasticsearch utilizando BM25.
    Retorna los documentos más relevantes y sus puntuaciones.
    
    Args:
        consulta (str): Términos de búsqueda.
        numero_resultados (int): Número de resultados a retornar.

    Returns:
        resultados (list): Lista de documentos relevantes con título y trama.
    """
    # Query DSL para realizar la búsqueda
    query = {
        "query": {
            "match": {
                "Plot": consulta  # Buscar coincidencias en la trama
            }
        },
        "size": numero_resultados
    }

    # Realizar la consulta
    respuesta = es.search(index=index_name, body=query)

    # Procesar resultados
    resultados = []
    for hit in respuesta['hits']['hits']:
        resultados.append({
            "Title": hit['_source']['Title'],
            "Plot": hit['_source']['Plot'],
            "Score": hit['_score']
        })
    
    return resultados


In [22]:
# Ejemplo de consulta con BM25
consulta_ejemplo = "dinosaurs"

# Obtener los documentos más relevantes
resultados_bm25 = buscar_con_bm25(consulta_ejemplo, numero_resultados=5)

# Mostrar los resultados
print(f"Resultados para la consulta: '{consulta_ejemplo}'\n")
for rank, resultado in enumerate(resultados_bm25, start=1):
    print(f"Rank {rank}")
    print(f"Título: {resultado['Title']}")
    print(f"Score: {resultado['Score']:.4f}")
    print(f"Trama (primeros 200 caracteres): {resultado['Plot'][:200]}...")
    print("-" * 50)


Resultados para la consulta: 'dinosaurs'

Rank 1
Título: We're Back! A Dinosaur's Story
Score: 12.8403
Trama (primeros 200 caracteres): In present-day New York City, an Eastern bluebird named Buster runs away from his siblings and he meets an intelligent orange Tyrannosaurus named Rex, who is playing golf. He explains to Buster that h...
--------------------------------------------------
Rank 2
Título: Theodore Rex
Score: 11.6915
Trama (primeros 200 caracteres): In an alternate futuristic society where humans and anthropomorphic dinosaurs co-exist, a tough police detective named Katie Coltraine (Whoopi Goldberg) is paired with a Tyrannosaurus named Theodore R...
--------------------------------------------------
Rank 3
Título: Future War
Score: 11.5795
Trama (primeros 200 caracteres): Future War begins aboard a spaceship undergoing a revolt. A man enters and activates an escape pod which travels to Earth and crashes into the Pacific Ocean. The pod contains “The Runaway”, a human sl...


## Reflexión Parte 2: Recuperación con BM25
Usamos Elasticsearch para implementar BM25, lo que permitió una recuperación más robusta que TF-IDF al considerar la saturación de términos y la longitud de los documentos. Esta técnica mostró mejores resultados al priorizar documentos más relevantes para consultas específicas como "dinosaurs".


# Parte 3: Recuperación con FAISS

#### **Parte 1: Preparación de datos**  
Esta etapa consiste en preparar los datos para su posterior procesamiento:  
- Se carga un dataset que contiene tramas y títulos de películas.  
- Se eliminan filas con valores nulos en la columna `Plot` para evitar errores.  
- Se crea una nueva columna, `texto_completo`, combinando el título y la trama de cada película. Esto asegura que toda la información relevante esté en un solo campo para la recuperación posterior.  

In [ ]:
# Parte 1: Preparación de datos
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from tqdm import tqdm

# Cargar el dataset
ruta_dataset = "wiki_movie_plots_deduped.csv"
df = pd.read_csv(ruta_dataset)

# Eliminar filas con valores nulos en la columna 'Plot'
df = df.dropna(subset=['Plot'])

# Crear la columna 'texto_completo' combinando 'Title' y 'Plot'
df['texto_completo'] = df['Title'].astype(str) + " " + df['Plot'].astype(str)

# Verificar que 'texto_completo' fue creado correctamente
print("Columnas disponibles:", df.columns)
print("Ejemplo de texto completo:", df['texto_completo'].head())

# Parte 2: Generación de embeddings con SentenceTransformer
modelo_embeddings = SentenceTransformer('all-MiniLM-L6-v2')  # Puedes usar otro modelo según necesidad

# Convertir los textos en vectores
textos = df['texto_completo'].tolist()

#### **Parte 2: Generación de embeddings con SentenceTransformer**  
En este paso, los datos procesados son convertidos a representaciones vectoriales llamadas embeddings:  
- Se utiliza el modelo `SentenceTransformer` para representar semánticamente los textos combinados.  
- El progreso de la generación de embeddings se muestra mediante una barra de carga (`tqdm`) para facilitar el monitoreo.  
- Estos embeddings serán usados más adelante para realizar búsquedas eficientes basadas en similitud.  


In [ ]:


# Generar embeddings y mostrar progreso
embeddings = []
for texto in tqdm(textos, desc="Generando embeddings"):
    embeddings.append(modelo_embeddings.encode(texto, convert_to_numpy=True))
embeddings = np.array(embeddings)

Columnas disponibles: Index(['Release Year', 'Title', 'Origin/Ethnicity', 'Director', 'Cast',
       'Genre', 'Wiki Page', 'Plot', 'texto_completo'],
      dtype='object')
Ejemplo de texto completo: 0    Kansas Saloon Smashers A bartender is working ...
1    Love by the Light of the Moon The moon, painte...
2    The Martyred Presidents The film, just over a ...
3    Terrible Teddy, the Grizzly King Lasting just ...
4    Jack and the Beanstalk The earliest known adap...
Name: texto_completo, dtype: object


Generando embeddings: 100%|██████████| 34886/34886 [36:17<00:00, 16.02it/s]  



Agregando embeddings al índice FAISS:


Indexando en FAISS: 100%|██████████| 34886/34886 [00:00<00:00, 135553.57it/s]



Índice FAISS creado con 34886 vectores.



#### **Parte 3: Configurar FAISS**  
Aquí se configura un índice FAISS para permitir búsquedas rápidas de similitud entre vectores:  
- Se utiliza la métrica L2 (distancia euclidiana cuadrada) para medir similitudes entre vectores.  
- Los embeddings generados previamente se añaden al índice en un proceso que muestra su progreso con una barra de carga (`tqdm`).  
- Una vez creado, el índice contiene todos los vectores necesarios para las búsquedas futuras.  


In [ ]:

# Parte 3: Configurar FAISS
# Crear el índice en FAISS (usamos L2 para búsqueda más rápida)
dimension = embeddings.shape[1]  # Dimensión de los vectores
indice_faiss = faiss.IndexFlatL2(dimension)

# Agregar los embeddings al índice con barra de progreso
print("\nAgregando embeddings al índice FAISS:")
for i in tqdm(range(len(embeddings)), desc="Indexando en FAISS"):
    indice_faiss.add(np.expand_dims(embeddings[i], axis=0))

print(f"\nÍndice FAISS creado con {indice_faiss.ntotal} vectores.")


#### **Parte 4: Realizar consultas en FAISS**  
En esta etapa, se ejecutan búsquedas en el índice FAISS:  
- Una consulta textual es convertida en un vector de embedding utilizando el mismo modelo de SentenceTransformer.  
- FAISS busca en su índice los vectores más cercanos al vector de la consulta y devuelve los resultados más relevantes.  
- Cada resultado incluye el título, la trama y una métrica de similitud (distancia).  

In [4]:
# Parte 4: Consultas en FAISS
def buscar_con_faiss(consulta, numero_resultados=5):
    """
    Realiza una consulta en FAISS buscando los vectores más cercanos.
    """
    # Convertir la consulta a un vector de embedding
    vector_consulta = modelo_embeddings.encode([consulta], convert_to_numpy=True)
    
    # Buscar los vecinos más cercanos en FAISS
    distancias, indices = indice_faiss.search(vector_consulta, numero_resultados)
    
    resultados = []
    for indice, distancia in zip(indices[0], distancias[0]):
        resultados.append({
            "Title": df.iloc[indice]['Title'],
            "Plot": df.iloc[indice]['Plot'],
            "Distancia": distancia
        })
    return resultados



#### **Parte 5: Comparar resultados de TF-IDF, BM25 y FAISS**  
En esta etapa, se comparan los resultados obtenidos utilizando las técnicas TF-IDF, BM25 y FAISS:  
- Para TF-IDF y BM25, se calculan las puntuaciones de relevancia de los documentos recuperados.  
- Para FAISS, se utiliza la métrica de distancia para identificar los documentos más cercanos.  
- Los cuatro primeros resultados de cada técnica son mostrados junto con sus métricas respectivas, permitiendo una comparación clara de su efectividad.  


In [5]:
# Ejemplo de consulta
consulta_ejemplo = "A park with cloned dinosaurs"
resultados_faiss = buscar_con_faiss(consulta_ejemplo, numero_resultados=5)

# Mostrar los resultados
print(f"\nResultados para la consulta: '{consulta_ejemplo}'\n")
for rank, resultado in enumerate(resultados_faiss, start=1):
    print(f"Rank {rank}")
    print(f"Título: {resultado['Title']}")
    print(f"Distancia: {resultado['Distancia']:.4f}")
    print(f"Trama (primeros 200 caracteres): {resultado['Plot'][:200]}...")
    print("-" * 50)


Resultados para la consulta: 'A park with cloned dinosaurs'

Rank 1
Título: Jurassic Park
Distancia: 0.6588
Trama (primeros 200 caracteres): Industrialist John Hammond and his bioengineering company, InGen, have created a theme park called Jurassic Park on Isla Nublar, a Costa Rican island, populated with cloned dinosaurs. After one of the...
--------------------------------------------------
Rank 2
Título: Dinosaurs! – A Fun-Filled Trip Back in Time!
Distancia: 1.0297
Trama (primeros 200 caracteres): The video—with beginning scenes filmed in 1987—begins with a young boy named Phillip (played by Fred Savage) sitting in his bedroom, listening to loud music, and struggling to find an idea for a class...
--------------------------------------------------
Rank 3
Título: Jurassic World
Distancia: 1.0311
Trama (primeros 200 caracteres): Brothers Zach and Gray Mitchell visit Isla Nublar, the site of the original Jurassic Park, where a new theme park named Jurassic World has operated for year


#### **Ejemplo de Consulta y Comparación**  
Se ejecuta una consulta específica, como "A park with cloned dinosaurs", y se compara el rendimiento de las tres técnicas.  
- Se presentan los títulos, tramas (truncadas para legibilidad) y métricas de relevancia para los cuatro mejores resultados obtenidos por cada técnica.  
- Esto permite evaluar cuál de las estrategias ofrece una recuperación más relevante para el usuario.

In [18]:
# Función para comparar resultados de TF-IDF, BM25 y FAISS
def comparar_resultados(consulta, numero_resultados=4):
    """
    Compara los resultados obtenidos con TF-IDF, BM25 y FAISS y muestra los 4 primeros de cada técnica.
    """
    print(f"\nComparación de resultados para la consulta: '{consulta}'\n")
    
    # TF-IDF
    indices_tfidf, puntuaciones_tfidf = buscar_con_tfidf(consulta, numero_resultados)
    resultados_tfidf = [
        {"Rank": rank + 1, 
         "Title": df.loc[indice, 'Title'], 
         "Score": puntuacion, 
         "Plot": df.loc[indice, 'Plot'][:200]}
        for rank, (indice, puntuacion) in enumerate(zip(indices_tfidf, puntuaciones_tfidf))
    ]
    print("Resultados con TF-IDF:")
    for resultado in resultados_tfidf:
        print(f"Rank {resultado['Rank']}: {resultado['Title']} (Score: {resultado['Score']:.4f})")
        print(f"Trama: {resultado['Plot']}...")
        print("-" * 50)
    
    # BM25
    resultados_bm25 = buscar_con_bm25(consulta, numero_resultados)
    print("\nResultados con BM25:")
    for rank, resultado in enumerate(resultados_bm25, start=1):
        print(f"Rank {rank}: {resultado['Title']} (Score: {resultado['Score']:.4f})")
        print(f"Trama: {resultado['Plot'][:200]}...")
        print("-" * 50)
    
    # FAISS
    resultados_faiss = buscar_con_faiss(consulta, numero_resultados)
    print("\nResultados con FAISS:")
    for rank, resultado in enumerate(resultados_faiss, start=1):
        print(f"Rank {rank}: {resultado['Title']} (Distancia: {resultado['Distancia']:.4f})")
        print(f"Trama: {resultado['Plot'][:200]}...")
        print("-" * 50)

# Ejemplo de comparación mostrando los 4 primeros resultados
consulta_ejemplo = "A park with cloned dinosaurs"
comparar_resultados(consulta_ejemplo, numero_resultados=4)



Comparación de resultados para la consulta: 'A park with cloned dinosaurs'

Resultados con TF-IDF:
Rank 1: Theodore Rex (Score: 0.2996)
Trama: In an alternate futuristic society where humans and anthropomorphic dinosaurs co-exist, a tough police detective named Katie Coltraine (Whoopi Goldberg) is paired with a Tyrannosaurus named Theodore R...
--------------------------------------------------
Rank 2: We're Back! A Dinosaur's Story (Score: 0.2416)
Trama: In present-day New York City, an Eastern bluebird named Buster runs away from his siblings and he meets an intelligent orange Tyrannosaurus named Rex, who is playing golf. He explains to Buster that h...
--------------------------------------------------
Rank 3: Jurassic Park (Score: 0.2376)
Trama: Industrialist John Hammond and his bioengineering company, InGen, have created a theme park called Jurassic Park on Isla Nublar, a Costa Rican island, populated with cloned dinosaurs. After one of the...
-----------------------------------

# Parte 4

1. Importacion y configuracion de ChromaDB
    -   Configura las bases para procesar un dataset de tramas de películas almacenado en un archivo CSV. Se utilizan bibliotecas como chromadb para manejar embeddings y bases de datos vectoriales, pandas para la manipulación de datos tabulares.

In [2]:
ruta_dataset = "wiki_movie_plots_deduped.csv"
import chromadb
from chromadb.utils import embedding_functions
import numpy as np
import pandas as pd
from tqdm import tqdm
import textwrap
import shutil
import os
import time
import gc

2. Función para limpiar ChromaDB
    - Se define funciones para gestionar y configurar la base de datos. La función cleanup_chromadb genera un directorio para almacenar datos. La función setup_chromadb inicializa un cliente de ChromaDB, se crea una colección específica para tramas de películas y asocia una función de embeddings predeterminada, asegurando que los datos estén organizados y preparados para análisis posteriores.

In [3]:
def cleanup_chromadb():
    persist_directory = f"chroma_db_{int(time.time())}"
    return persist_directory

def setup_chromadb():
    persist_directory = cleanup_chromadb()
    
    try:
        client = chromadb.PersistentClient(path=persist_directory)
        
        embedding_function = embedding_functions.DefaultEmbeddingFunction()
        
        collection = client.create_collection(
            name="movies_collection",
            embedding_function=embedding_function,
            metadata={"description": "Colección de películas de Wikipedia con sus tramas"}
        )
        print(f"Nueva colección creada en {persist_directory}")
        
        return collection
    
    except Exception as e:
        print(f"Error al configurar ChromaDB: {str(e)}")
        raise

3. Cargar dataset y se procesa
    - load_movie_dataset: Carga el CSV, limpia valores nulos y permite trabajar.
    - process_batch: Convierte un lote de datos de películas en IDs, tramas y metadatos organizados para ser insertados en la colección.
    - insert_movies_batch: Divide el dataset en lotes pequeños para insertarlos eficientemente en ChromaDB, manejando memoria y errores durante el proceso.

In [4]:
def load_movie_dataset(ruta_dataset, num_samples=None):
    try:
        columns = ['Title', 'Plot', 'Release Year', 'Director', 'Genre', 'Origin/Ethnicity']
        df = pd.read_csv(ruta_dataset, usecols=columns)
        print(f"Dataset cargado exitosamente. Dimensiones: {df.shape}")
        
        df = df.fillna('')
        
        if num_samples:
            df = df.sample(n=min(num_samples, len(df)), random_state=42)
            print(f"Muestra seleccionada: {len(df)} películas")
        
        gc.collect()
        
        return df
    except Exception as e:
        print(f"Error al cargar el dataset: {str(e)}")
        raise

def process_batch(batch_df):
    ids = [f"movie_{idx}" for idx in batch_df.index]
    plots = batch_df['Plot'].astype(str).tolist()
    
    metadatas = batch_df.apply(
        lambda row: {
            "title": str(row['Title']),
            "year": str(row['Release Year']),
            "director": str(row['Director']),
            "genre": str(row['Genre']),
            "origin": str(row['Origin/Ethnicity'])
        }, 
        axis=1
    ).tolist()
    
    return ids, plots, metadatas

def insert_movies_batch(collection, df, batch_size=10):
    total_batches = len(df) // batch_size + (1 if len(df) % batch_size != 0 else 0)
    
    for i in tqdm(range(total_batches), desc="Insertando películas"):
        try:
            start_idx = i * batch_size
            end_idx = min((i + 1) * batch_size, len(df))
            batch_df = df.iloc[start_idx:end_idx].copy()
            
            ids, plots, metadatas = process_batch(batch_df)
            
            collection.add(
                ids=ids,
                documents=plots,
                metadatas=metadatas
            )
            
            del batch_df
            gc.collect()
            time.sleep(0.1)
            
        except Exception as e:
            print(f"Error en el lote {i}: {str(e)}")
            continue

4. Consultar peliculas similares
    - La función query_similar_movies permite realizar búsquedas de películas similares utilizando texto como consulta. Utiliza la colección de ChromaDB para encontrar resultados relevantes basados en embeddings, retornando detalles como el título, año, director, género, origen, trama y la distancia de similitud.

In [5]:
def query_similar_movies(collection, query_text, n_results=5):
    try:
        results = collection.query(
            query_texts=[query_text],
            n_results=n_results
        )
        
        similar_movies = []
        for i in range(len(results['documents'][0])):
            metadata = results['metadatas'][0][i]
            plot = textwrap.fill(results['documents'][0][i], width=80)
            
            similar_movies.append({
                'title': metadata['title'],
                'year': metadata['year'],
                'director': metadata['director'],
                'genre': metadata['genre'],
                'origin': metadata['origin'],
                'plot': plot,
                'distance': results['distances'][0][i]
            })
        
        return similar_movies
    except Exception as e:
        print(f"Error en la búsqueda: {str(e)}")
        return []

In [9]:
print("Cargando dataset...")
df = load_movie_dataset(ruta_dataset, num_samples=50)  

# 2. Configurar ChromaDB
print("Configurando ChromaDB...")
collection = setup_chromadb()

# 3. Insertar películas
print("Insertando películas en ChromaDB...")
insert_movies_batch(collection, df, batch_size=10)  

# 4. Ejemplo de consulta
print("\nRealizando consulta de ejemplo...")
query = "A park with cloned dinosaurs"
similar_movies = query_similar_movies(collection, query)

# 5. Mostrar resultados
if similar_movies:
    print("\nPelículas similares encontradas:")
    for movie in similar_movies:
        print("\n" + "="*80)
        print(f"Título: {movie['title']} ({movie['year']})")
        print(f"Director: {movie['director']}")
        print(f"Género: {movie['genre']}")
        print(f"Origen: {movie['origin']}")
        print(f"Distancia: {movie['distance']:.4f}")
        print("\nTrama:")
        print(movie['plot'])
else:
    print("No se encontraron resultados")

Cargando dataset...
Dataset cargado exitosamente. Dimensiones: (34886, 6)
Muestra seleccionada: 50 películas
Configurando ChromaDB...
Nueva colección creada en chroma_db_1736896840
Insertando películas en ChromaDB...


Insertando películas: 100%|██████████| 5/5 [00:02<00:00,  2.29it/s]


Realizando consulta de ejemplo...

Películas similares encontradas:

Título: The Burning (1981)
Director: Tony Maylam
Género: horror
Origen: American
Distancia: 1.4773

Trama:
One night at Camp Blackfoot, several campers pull a prank on the caretaker named
Cropsy by setting a worm-riddled skull next to his bed with candles in the eye
sockets. When the caretaker is awoken by the campers banging on his window, he
is frightened by the skull and accidentally knocks it onto his bed, starting a
fire. The flames reach to a gas tank, where it ignites Cropsy and his cabin. He
runs outside, engulfed in flames, and stumbles down into a river as the boys
flee from the scene.  Five years later, Cropsy is released from the hospital
despite having to deal with failed skin grafts, and wears a coat and hat to hide
his deformities. While outside, a prostitute lures him, and in a fit of rage, he
repeatedly stabs her with a pair of scissors. Soon, he arms himself with a pair
of garden shears and sets out

6. Evaluar los resultados:
    * Compara los documentos recuperados por ChromaDB con los de FAISS, TF-IDF y BM25.

#### Resultados: 

* Resultados con TF-IDF:
    - Rank 1: Theodore Rex (Score: 0.2996)
    - Rank 2: We're Back! A Dinosaur's Story (Score: 0.2416)
    - Rank 3: Jurassic Park (Score: 0.2376)
    - Rank 4: Dinosaurs! – A Fun-Filled Trip Back in Time! (Score: 0.2237)

* Resultados con BM25:
    - Rank 1: Jurassic Park (Score: 22.3337)
    - Rank 2: The Lost World: Jurassic Park (Score: 15.8991)
    - Rank 3: We're Back! A Dinosaur's Story (Score: 15.1359)
    - Rank 4: Jurassic World (Score: 14.8756)

* Resultados con FAISS:
    - Rank 1: Jurassic Park (Distancia: 0.6588)
    - Rank 2: Dinosaurs! – A Fun-Filled Trip Back in Time! (Distancia: 1.0297)
    - Rank 3: Jurassic World (Distancia: 1.0311)
    - Rank 4: Jurassic Park III (Distancia: 1.0853)

* Resultados con Chroma DB:
    - Rank 1: Jurassic Park (Distancia: 0.6588)
    - Rank 2: Dinosaurs! – A Fun-Filled Trip Back in Time! (Distancia: 1.0297)
    - Rank 3: Jurassic World (Distancia: 1.0311)
    - Rank 4: Jurassic Park III (Distancia: 1.0853)

#### Conclusión:
- ChromaDB se destaca por su capacidad para recuperar documentos con significados más profundos y semánticos, utilizando embeddings. Aunque introduce mayor diversidad en los resultados, como se observa con películas que, aunque no directamente relacionadas con la consulta, exploran conexiones narrativas o conceptuales, lo que puede incluir títulos como "The Burning" o "Mother Lode". 
- En comparación con otras técnicas como TF-IDF y BM25, ChromaDB ofrece una visión más amplia y abstracta, aunque su precisión depende del tipo de consulta.

# Parte 5: Comparación de Resultados


### Relevancia:

La comparación entre las diferentes técnicas de recuperación de información, **TF-IDF**, **BM25**, **FAISS** y **ChromaDB**, muestra variaciones en su capacidad para identificar documentos relevantes para la consulta. Cada uno de estos enfoques tiene características únicas que impactan los resultados obtenidos:

- **TF-IDF**:  
  La estrategia basada en TF-IDF tiene un enfoque clásico que se basa en la frecuencia de términos y la importancia relativa de las palabras dentro de los documentos. Aunque ofrece buenos resultados cuando se busca una coincidencia exacta de términos, su rendimiento en consultas más complejas se ve limitado. Esto se observa en el hecho de que las puntuaciones altas, están algo alejados del tema central, lo que refleja la dependencia de TF-IDF de la frecuencia de términos en lugar de la comprensión contextual.

- **BM25**:  
  BM25, por su parte, ajusta la influencia de los términos según la longitud del documento y la frecuencia de aparición de los términos, lo que le otorga una mayor precisión en la recuperación de documentos. Los resultados obtenidos, son claramente más relevantes para el tema de la consulta y reflejan la capacidad de BM25 para manejar consultas más complejas y de mayor longitud.

- **FAISS y ChromaDB**:  
  Ambas tecnologías basadas en **embeddings** y similitudes semánticas destacan por su capacidad para comprender el contexto de una consulta y recuperar documentos relacionados no solo por términos exactos, sino también por similitud conceptual. Esto se refleja en la coincidencia de resultados, que están estrechamente relacionados con la temática, proporcionando resultados más precisos y semánticamente relevantes en comparación con TF-IDF y BM25.

### Ventajas y Limitaciones:

#### **TF-IDF**:
- **Ventajas**:
  - Sencillez y eficiencia computacional en textos de tamaño pequeño o mediano.
  - Rápido en consultas directas con coincidencia exacta de términos.
- **Limitaciones**:
  - No captura relaciones semánticas ni contextuales profundas.
  - Menos eficiente en consultas abstractas o con variaciones lingüísticas.

#### **BM25**:
- **Ventajas**:
  - Más sofisticado que TF-IDF al manejar documentos largos y consultas complejas.
  - Ajusta la relevancia considerando la longitud del documento y la frecuencia de términos.
- **Limitaciones**:
  - A pesar de ser más efectivo que TF-IDF, sigue siendo una técnica basada en palabras clave, sin capacidad para entender relaciones semánticas profundas.

#### **FAISS**:
- **Ventajas**:
  - Utiliza vectores de alta dimensionalidad y espacio de similitud, lo que permite capturar contextos y significados más complejos.
  - Resultados más precisos en consultas que requieren una comprensión contextual de los términos.
- **Limitaciones**:
  - Puede ser menos preciso cuando los documentos están muy lejos semánticamente pero contienen palabras clave similares.
  - Requiere un entrenamiento previo de los embeddings y más recursos computacionales.

#### **ChromaDB**:
- **Ventajas**:
  - Similar a FAISS, pero con un enfoque más optimizado en la gestión de vectores y la indexación para consultas de alta carga.
  - Ideal para consultas complejas que requieren inferencia semántica más allá de las palabras clave.
- **Limitaciones**:
  - Similar a FAISS en cuanto a requerir recursos significativos para procesar consultas y almacenar embeddings.
  - En algunos casos, puede ser demasiado flexible y devolver resultados menos relevantes si no se configura adecuadamente.
  
### Conclusión:

De acuerdo con los resultados obtenidos, **FAISS** y **ChromaDB** muestran un rendimiento superior al recuperar documentos que son semánticamente relevantes para la consulta. Sin embargo, **BM25** y **TF-IDF** siguen siendo opciones válidas dependiendo del tipo de consulta, ya que las técnicas basadas en palabras clave pueden ser más rápidas y eficientes en ciertos contextos.

### Resultados:

#### Query: 
  - "A park with cloned dinosaurs"

#### **TF-IDF**:
- **Rank 1**: Theodore Rex (Score: 0.2996)  
  **Trama**: Un dinosaurio robótico se convierte en el compañero de un policía.
- **Rank 2**: We're Back! A Dinosaur's Story (Score: 0.2416)  
  **Trama**: Cuatro dinosaurios viajan al presente para tener una nueva vida en Nueva York.
- **Rank 3**: Jurassic Park (Score: 0.2376)  
  **Trama**: Un parque temático de dinosaurios es creado en una isla remota.
- **Rank 4**: Dinosaurs! – A Fun-Filled Trip Back in Time! (Score: 0.2237)  
  **Trama**: Un niño viaja al pasado para explorar la era de los dinosaurios.

#### **BM25**:
- **Rank 1**: Jurassic Park (Score: 22.3337)  
  **Trama**: Un parque de dinosaurios es creado por una compañía de bioingeniería.
- **Rank 2**: The Lost World: Jurassic Park (Score: 15.8991)  
  **Trama**: Un equipo explora una isla llena de dinosaurios salvajes.
- **Rank 3**: We're Back! A Dinosaur's Story (Score: 15.1359)  
  **Trama**: Cuatro dinosaurios vuelven al presente para una nueva aventura.
- **Rank 4**: Jurassic World (Score: 14.8756)  
  **Trama**: Un nuevo parque temático de dinosaurios enfrenta problemas graves.

#### **FAISS**:
- **Rank 1**: Jurassic Park (Distancia: 0.6588)  
  **Trama**: El parque de dinosaurios en Isla Nublar enfrenta un desastre cuando los animales escapan.
- **Rank 2**: Dinosaurs! – A Fun-Filled Trip Back in Time! (Distancia: 1.0297)  
  **Trama**: Un niño viaja al pasado para ver dinosaurios en su hábitat natural.
- **Rank 3**: Jurassic World (Distancia: 1.0311)  
  **Trama**: Un nuevo parque de dinosaurios se abre, pero los problemas surgen cuando los visitantes son atacados.
- **Rank 4**: Jurassic Park III (Distancia: 1.0853)  
  **Trama**: Un grupo de personas intenta rescatar a un niño perdido en Isla Sorna.

#### **ChromaDB**:
- **Rank 1**: Jurassic Park (Distancia: 0.6588)  
  **Trama**: Un parque temático con dinosaurios clonados enfrenta problemas cuando los animales escapan.
- **Rank 2**: Dinosaurs! – A Fun-Filled Trip Back in Time! (Distancia: 1.0297)  
  **Trama**: Un niño viaja al pasado para observar dinosaurios en su entorno natural.
- **Rank 3**: Jurassic World (Distancia: 1.0311)  
  **Trama**: El parque temático Jurassic World enfrenta una crisis cuando sus dinosaurios se sueltan.
- **Rank 4**: Jurassic Park III (Distancia: 1.0853)  
  **Trama**: Un niño es rescatado de una isla llena de dinosaurios.
